In [ ]:
import json
import requests
import urllib.parse

import pandas as pd
from sqlalchemy import create_engine

## xAPIステートメントを取得

<span style="color:red"><b>Learning Lockerで作成したLRSのクライアント情報を設定すること。</b></span>

In [ ]:
user = '<Key>'
passwd = '<Secret>'

[AGGREGATION HTTP INTERFACE](https://learninglocker.atlassian.net/wiki/spaces/DOCS/pages/106037259/Aggregation+API)で検索条件( `$match` )と件数( `$limit` )を指定し、ステートメントを取得する。  
ステートメントの詳細は[xAPI仕様](https://github.com/adlnet/xAPI-Spec)([日本語版](https://github.com/elc-gh/xAPI-Spec_ja))を参照すること。

In [ ]:
pipeline = urllib.parse.quote(json.dumps([
    {'$project': {
        'timestamp': 1,
        'statement': 1,
        '_id': 0
    }},
    # 検索条件
    {'$match': {
        #'statement.actor.name': '...', # ユーザの絞り込み
        #'statement.verb.display.en': '...', # 操作の絞り込み
        'timestamp': { # 操作日時の絞り込み
            "$gte": {
                '$dte': '2022-04-01T00:00:00Z'
            },
            "$lte": {
                '$dte': '9999-12-31T00:00:00Z'
            }
        }
    }},
    # 取得する最大件数
    {'$limit': 30000},
]))
url = f'http://learninglocker:8080/statements/aggregate?cache=false&pipeline={pipeline}'

r = requests.get(url, auth=(user, passwd))

# Jsonを辞書型に変換
response_data = json.loads(r.text)
# 辞書からステートメントのみを抽出
stats = [d['statement'] for d in response_data]
# ネスト項目をフラットにしてpandasのDataFrameを作成
df = pd.io.json.json_normalize(stats)
pd.set_option('display.max_columns', 100)
df.head()

## ステートメントの分析
分析結果は最終的にpandasのDataframe形式とすること

In [ ]:
# 任意の分析処理
# ...

In [ ]:
# Supersetで可視化するデータを用意
df = df[['object.definition.description.en', 'object.id', 'timestamp']]
df.head()

## Supersetに処理結果を登録

In [ ]:
engine = create_engine('postgresql://postgres@superset-db:5432/jupyter')

# 登録するテーブルの名前を指定
tablename = 'notebook_results'

# Supersetにテーブルを登録
from sqlalchemy.types import TIMESTAMP
df.to_sql(
    tablename,
    engine,
    if_exists='replace',
    dtype={'timestamp': TIMESTAMP()}  # 必要に応じてdtypeでカラムの属性を指定する
)

s = requests.Session()
r = s.post(
    'http://superset:8088/api/v1/security/login',
    json={
        'username': 'admin',
        'password': 'admin',
        'provider': 'db', 
    }
)
headers={
    'Authorization': f'Bearer {r.json()["access_token"]}'
}

r = s.get(
    'http://superset:8088/api/v1/database?q=(filters:!((col:database_name,opr:eq,value:Jupyter)))',
    headers=headers,
)
database_id = r.json()['ids'][0]

r = s.get(
    'http://superset:8088/api/v1/security/csrf_token',
    headers=headers
)
headers['X-CSRFToken'] = r.json()['result']

r = s.post(
    'http://superset:8088/api/v1/dataset',
    headers=headers,
    json={
        'database': database_id,
        'schema': 'public',
        'table_name': tablename
    }
)
print(r.json())

## Supersetに登録済みのテーブル一覧

In [ ]:
r = s.get(
    'http://superset:8088/api/v1/dataset',
    headers=headers
)
for dataset in r.json()['result']:
    print(dataset['table_name'])